In [1]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2023-05-08 09:41:59--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 108.139.75.108, 108.139.75.124, 108.139.75.98, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|108.139.75.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: 'phix.fa.1'

     0K .....                                                 100%  504K=0.01s

2023-05-08 09:41:59 (504 KB/s) - 'phix.fa.1' saved [5528/5528]



In [5]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [6]:
genome = readGenome('phix.fa')

In [7]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences

In [15]:
t = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p = 'AG'
naive(p, t)

[27, 38, 44]

In [16]:
import random
def generateReads(genome, numReads, readLen):
    ''' Generate reads from random positions in the given genome. '''
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome)-readLen) - 1
        reads.append(genome[start : start+readLen])
    return reads

In [17]:
# Generate 100 reads of length 100
reads = generateReads(genome, 100, 100)

# Count how many reads match the genome exactly
numMatched = 0
for r in reads:
    matches = naive(r, genome)
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, len(reads)))

100 / 100 reads matched the genome exactly!


In [1]:
#!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2023-05-25 08:32:35--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 108.139.75.98, 108.139.75.124, 108.139.75.108, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|108.139.75.98|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: 'chr1.GRCh38.excerpt.fasta'

     0K .......... .......... .......... .......... ..........  6%  197K 4s
    50K .......... .......... .......... .......... .......... 12%  265K 3s
   100K .......... .......... .......... .......... .......... 18%  321K 3s
   150K .......... .......... .......... .......... .......... 25%  176K 3s
   200K .......... .......... .......... .......... .......... 31%  735K 2s
   250K .......... .......... .......... .......... .......... 37% 73.9M 2s
   300K .......... .......... .......... .......... .......... 44%  796K 1s


In [18]:
def readFastq(filename):
    sequences = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            fh.readline() # skip base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences

In [19]:
import collections
# phix_reads = readFastq('ERR266411_1.first1000.fastq')
phix_reads = readFastq('chr1.GRCh38.excerpt.fasta')
count = collections.Counter()
for read in phix_reads:
    count.update(read)
count

Counter({'T': 64923, 'G': 36386, 'A': 63304, 'C': 35387})

In [20]:
numMatched = 0
n = 0
for r in phix_reads: # phix_reads is the list of reads from the fastq file
    matches = naive(r, genome) # naive is the function defined above in this notebook
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

0 / 2500 reads matched the genome exactly!


In [12]:
# Now let's try matching just the first 30 bases of each read
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

0 / 2500 reads matched the genome exactly!


In [13]:
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [14]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    matches.extend(naive(reverseComplement(r), genome))
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

0 / 2500 reads matched the genome exactly!
